In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/kaggle.json ~/
! mkdir ~/.kaggle
! mv ~/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d eldarsultanow/ml-bsd-dataset

ml-bsd-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip ml-bsd-dataset.zip

Archive:  ml-bsd-dataset.zip
replace rank-vs-ap.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: rank-vs-ap.csv          


In [ ]:
!pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
import csv
import fileinput
from collections import defaultdict
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from supervised.automl import AutoML

In [ ]:
df = pd.read_csv("rank-vs-ap.csv", index_col=False)

In [ ]:
df.head()

,rank,ap_1,ap_2,ap_3,ap_4,ap_5,ap_6,ap_7,ap_8,ap_9,...,ap_1218,ap_1219,ap_1220,ap_1221,ap_1222,ap_1223,ap_1224,ap_1225,ap_1226,ap_1227
0,1,0.707107,0.000000,0.000000,-0.755929,1.809068,-1.109400,0.000000,0.917663,-0.417029,...,-0.553582,-1.116551,-1.196781,-0.974837,-1.707961,-0.311200,-1.696032,-0.762636,-0.371096,0.421075
1,1,0.000000,1.154701,0.000000,-0.377964,0.000000,0.554700,0.727607,1.835326,1.668115,...,0.624038,-1.186964,1.729801,-1.869276,0.934355,-1.686506,0.662356,1.204162,-0.742193,-0.781997
2,1,0.707107,0.000000,-1.341641,0.755929,-1.206045,-1.664101,-0.242536,0.917663,-0.625543,...,0.312019,-1.599384,1.035869,1.276334,-0.974542,-0.240929,1.224354,-0.672324,0.671985,0.290742
3,0,-0.707107,0.000000,0.000000,1.889822,-0.301511,0.554700,-0.970143,1.147079,1.251086,...,-0.161042,-1.850859,1.387864,-0.291446,-0.341592,0.090349,-0.832962,0.642219,1.404148,1.022611
4,1,-0.707107,-1.154701,0.894427,-0.377964,0.000000,-0.554700,-0.970143,1.835326,0.000000,...,-1.892245,-1.549089,-0.100570,-0.422095,0.713325,1.023950,-1.264497,0.070243,-1.514474,0.401024


In [ ]:
#ap_1 to ap_1227
max = 1227
col_train = range(1,max+1)
col_predict = 0

x_var_2d = df.iloc[:, col_train]
y_var = df.iloc[:, col_predict]
(X_train, X_test, y_train, y_test) = train_test_split(x_var_2d, y_var, test_size=1/3, random_state=42)

In [ ]:
#"Xgboost", "LightGBM", "Neural Network", "Nearest Neighbors", "CatBoost"
automl = AutoML(
    algorithms=["Xgboost", "LightGBM", "Neural Network", "Nearest Neighbors", "CatBoost"],
    mode="Explain",
    explain_level = 2,
    golden_features=True,
    stack_models=False,
    train_ensemble=False,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    })

automl.fit(X_train, y_train)

AutoML directory: AutoML_1
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Xgboost', 'LightGBM', 'Neural Network', 'Nearest Neighbors', 'CatBoost']
AutoML steps: ['simple_algorithms', 'default_algorithms', 'golden_features']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
y_true and y_pred contain different number of classes 6, 7. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5]
Problem during computing permutation importance. Skipping ...


/usr/local/lib/python3.9/dist-packages/supervised/utils/shap.py:39: UserWarning: Disable SHAP explanations because of number of columns > 500.


y_true and y_pred contain different number of classes 6, 7. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5]
Problem during computing permutation importance. Skipping ...


/usr/local/lib/python3.9/dist-packages/supervised/utils/shap.py:39: UserWarning: Disable SHAP explanations because of number of columns > 500.


y_true and y_pred contain different number of classes 6, 7. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5]
Problem during computing permutation importance. Skipping ...


/usr/local/lib/python3.9/dist-packages/supervised/utils/shap.py:39: UserWarning: Disable SHAP explanations because of number of columns > 500.


y_true and y_pred contain different number of classes 6, 7. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5]
Problem during computing permutation importance. Skipping ...


/usr/local/lib/python3.9/dist-packages/supervised/utils/shap.py:39: UserWarning: Disable SHAP explanations because of number of columns > 500.


1_Default_LightGBM logloss 1.263911 trained in 2916.87 seconds
2_Default_Xgboost not trained. Stop training after the first fold. Time needed to train on the first fold 5736.0 seconds. The time estimate for training on all folds is larger than total_time_limit.
Skip golden_features because no parameters were generated.
AutoML fit time: 8840.08 seconds
AutoML best model: 1_Default_LightGBM


AutoML(algorithms=['Xgboost', 'LightGBM', 'Neural Network', 'Nearest Neighbors',
                   'CatBoost'],
       explain_level=2, golden_features=True, stack_models=False,
       train_ensemble=False,
       validation_strategy={'k_folds': 4, 'shuffle': False, 'stratify': True,
                            'validation_type': 'kfold'})

In [ ]:
automl.report()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Conv1D, Input, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.python.keras.models import Sequential

In [ ]:
#https://groups.google.com/g/keras-users/c/YxjOWLXbUJk
#https://stackoverflow.com/questions/51849347/keras-and-input-shape-to-conv1d-issues/51854675#51854675
#https://victorzhou.com/blog/keras-cnn-tutorial/
#https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D

num_classes = df['rank'].nunique()

n_filters=32
kernel_size=3

model = Sequential()
model.add(Conv1D(n_filters, kernel_size, padding='valid', input_shape=(max, num_classes), activation='relu'))

model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))
model.add(Dropout(0.2))
model.add(Conv1D(n_filters, kernel_size, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

#model.add(MaxPooling1D(pool_size=5))
#model.add(Flatten())
#model.add(Dense(1000, activation='relu'))
#model.add(Dense(num_classes, activation='softmax'))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 1225, 32)          704       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 612, 32)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 612, 32)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 610, 32)           3104      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 305, 32)           0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 305, 32)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 305, 1)           

In [ ]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
svc_lin = svm.LinearSVC()
svc_nu = svm.NuSVC(gamma='auto')
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1, coef0=1, cache_size=300)
svr_poly.fit(X_train, y_train)

score = svr_poly.score(X_test, y_test)
print(score)

In [ ]:
ADBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10).fit(X_train, y_train)
print(ADBoost.score(X_test, y_test))

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.


0.33266
